In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import mutual_info_regression 

In [ ]:
df=pd.read_csv('lonal_entro.csv')
df.columns

In [ ]:
features_o = ['far_hdb_o', 'far_priv_o', 'far_comm_o']
features_d = ['far_hdb_d', 'far_priv_d', 'far_comm_d']
features_od=['far_hdb_o', 'far_priv_o', 'far_comm_o','far_hdb_d', 'far_priv_d', 'far_comm_d']
X = df.copy()
y = X.pop('trip_m6_wd_pm')
Xo = X[features_o]
Xd = X[features_d]
Xod = X[features_od]




In [ ]:
delcol=[ 'Unnamed: 0', 'Unnamed: 0.1','smooth_entro_o',
       'smooth_entro_d','ori_lat', 'ori_lon', 'dst_lat', 'dst_lon',
       'has_trip', 'ori_lonal_pc1', 'ori_lonal_pc2', 'dst_lonal_pc1',
       'dst_lonal_pc2', 'od_lonal_pc1', 'od_lonal_pc2', 'od_lonal_pc3',
       'od_lonal_pc4', 'lonal_baseline', 'ori_cluster', 'dst_cluster']
for col in delcol:
    try:
        del df[col]
    except:
        pass
df.to_csv('pca_far.csv')

In [ ]:
df.columns

In [ ]:
Xo.describe()

In [ ]:
# Standardize
Xo_scaled = (Xo - Xo.mean(axis=0)) / Xo.std(axis=0)
Xd_scaled = (Xd - Xd.mean(axis=0)) / Xd.std(axis=0)
Xod_scaled = (Xod - Xod.mean(axis=0)) / Xod.std(axis=0)

In [ ]:
from sklearn.decomposition import PCA

# Create principal components
pca = PCA()
Xo_pca = pca.fit_transform(Xo_scaled)
Xd_pca = pca.fit_transform(Xd_scaled)
Xod_pca = pca.fit_transform(Xod_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(Xo_pca.shape[1])]
Xo_pca = pd.DataFrame(Xo_pca, columns=component_names)
Xd_pca = pd.DataFrame(Xd_pca, columns=component_names)
Xod_pca = pd.DataFrame(Xod_pca, columns=['PC1','PC2','PC3','PC4','PC5','PC6'])

df['ori_far_pc1']=Xo_pca['PC1']
df['ori_far_pc2']=Xo_pca['PC2']
df['ori_far_pc3']=Xod_pca['PC3']
df['dst_far_pc1']=Xd_pca['PC1']
df['dst_far_pc2']=Xd_pca['PC2']
df['dst_far_pc3']=Xod_pca['PC3']
df['od_far_pc1']=Xod_pca['PC1']
df['od_far_pc2']=Xod_pca['PC2']
df['od_far_pc3']=Xod_pca['PC3']


In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
plt.style.use("bmh")
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
ycol='trip_m6_wd_pm'
xcol=[ 'entro_o', 'far_hdb_o',
       'far_priv_o', 'far_comm_o', 'entro_d', 
       'far_hdb_d', 'far_priv_d', 'far_comm_d', 'ori_far_pc1', 'ori_far_pc2',
       'ori_far_pc3', 'dst_far_pc2', 'od_far_pc1', 'od_far_pc2', 'od_far_pc3',
       'dst_far_pc1', 'dst_far_pc3']
y=df[ycol]
X=df[xcol]                                
scores=make_mi_scores(X, y)
plt.figure(dpi=200)
plot_mi_scores(scores)
plt.savefig('far_pca.jpg')

In [ ]:
df['ori_far_pc1']

In [ ]:
df.to_csv('pca_far.csv')